In [1]:
class Mapper:
    def __init__(self, map_triples):
        self.dest, self.source, self.length = [], [], []
        map_triples = sorted(map_triples, key=lambda x: x[1])
        for triple in map_triples:
            self.dest.append(triple[0])
            self.source.append(triple[1])
            self.length.append(triple[2])
        self.n_maps = len(map_triples)
        
    def __call__(self, item, return_id=False):
        for i in range(self.n_maps):
            source, dest, length = self.source[i], self.dest[i], self.length[i]
            if item >= source and item < (source + length):
                if return_id:
                    return i
                return dest + (item - source)
        if return_id:
            return -1
        return item
    
    def map_interval(self, start, length):
        new_intervals = []
        while True:
            i = self(start, return_id=True)
            if i != -1:
                # the start is inside an inverval
                source_start, dest_start, map_length = \
                    self.source[i], self.dest[i], self.length[i]
                if start + length - 1 < source_start + map_length:
                    # the end is inside the same interval
                    new_intervals.append((dest_start + (start - source_start), length))
                    return new_intervals
                else:
                    # the end is past the end of this interval
                    new_intervals.append((dest_start + (start - source_start), 
                                          source_start + map_length - start))
                    # reset and we start again
                    start = source_start + map_length
                    length = length - new_intervals[-1][-1]
                
            else: # the start is outside an interval
                # find the first interval
                found = False
                for i in range(self.n_maps):
                    if start < self.source[i]: # this is the first interval after
                        # process the default map
                        if start+length-1 < self.source[i]:
                            new_intervals.append((start, length))
                            return new_intervals
                        else:
                            new_intervals.append((start, self.source[i] - start))
                            start = self.source[i]
                            length = length - new_intervals[-1][-1]
                            found = True
                            break
                # we didn't find one
                if not found:
                    new_intervals.append((start, length))
                    return new_intervals
                    
                
                    

In [2]:
map_chain = {}
source_name, dest_name, triples = '','',[]
with open('input.txt') as fl:
    lns = fl.readlines()
    seeds = [int(n) for n in lns[0].split(':')[1].strip().split(' ')]
    for ln in lns[1:]:
        ln = ln.strip()
        if len(ln) > 0:
            vals = ln.split(' ')
            if not vals[0].isnumeric():
                # store the previous
                map_chain[source_name] = (dest_name, Mapper(triples))
                # setup the next
                names = vals[0].split('-')
                source_name, dest_name = names[0], names[2]
                triples = []
            else:
                triples.append(tuple(int(n) for n in vals))
    map_chain[source_name] = (dest_name, Mapper(triples))
    del map_chain['']
                

In [3]:
def map_seed(seed, map_chain=map_chain):
    item = seed
    source_name = 'seed'
    while True:
        try:
            dest_name, mapper = map_chain[source_name]
        except KeyError:
            return item
        item = mapper(item)
        source_name = dest_name
        
    

In [4]:
# part 1
min([map_seed(seed) for seed in seeds])

910845529

In [5]:
# part 2
seed_intervals = [(seeds[i], seeds[i+1]) for i in range(0, len(seeds), 2)]
new_intervals = seed_intervals
source_name = 'seed'
while True:
    try:
        dest_name, mapper = map_chain[source_name]
    except KeyError:
        break
    mapped_intervals = [mapper.map_interval(i[0],i[1]) for i in new_intervals]
    new_intervals = mapped_intervals[0]
    for i in mapped_intervals[1:]:
        new_intervals.extend(i)
    
    source_name = dest_name

In [6]:
min([i[0] for i in new_intervals])

77435348